[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alesaccoia/IULM_DDM2324_Notebooks/blob/main/08_associazione_variabili_categoriche.ipynb)

# Misurare l'associazione fra variabili categoriche

In [9]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats.contingency import margins

!wget "https://github.com/LeoLin72/IULM_DDM2324_Notebooks/raw/main/data/chisquare_preferenza_genere_prodotto.xlsx"

# Carico i dati dal file Excel, specificando quale worksheet
df = pd.read_excel('chisquare_preferenza_genere_prodotto.xlsx', sheet_name="Data")


--2024-04-09 13:23:08--  https://github.com/LeoLin72/IULM_DDM2324_Notebooks/raw/main/data/chisquare_preferenza_genere_prodotto.xlsx
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LeoLin72/IULM_DDM2324_Notebooks/main/data/chisquare_preferenza_genere_prodotto.xlsx [following]
--2024-04-09 13:23:08--  https://raw.githubusercontent.com/LeoLin72/IULM_DDM2324_Notebooks/main/data/chisquare_preferenza_genere_prodotto.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49025 (48K) [application/octet-stream]
Saving to: ‘chisquare_preferenza_genere_prodotto.xlsx.1’

chisquare_preferenz 100%[=============

## Tabella di contingenza

In [10]:
# Tabelle di contingenza:
contingency_table = pd.crosstab(df['gender'], df['productid'], margins=True, margins_name="Totale")
print(contingency_table)

productid    1     2    3  Totale
gender                           
1          495   590  272    1357
2          330   498  265    1093
Totale     825  1088  537    2450


## Probabilità Marginale
È la probabilità di un singolo evento senza considerare l'effetto di un altro evento. Si tratta essenzialmente della probabilità "a margine" di un evento particolare, quindi senza condizionarlo su un altro.
Ad esempio, considera una tabella di contingenza tra Genere (Maschio/Femmina) e Prodotto (1/2/3). La probabilità marginale che una persona scelta a caso sia un maschio (indipendentemente dal fatto che fumi o meno) si calcola sommando tutte le osservazioni dei maschi e dividendo per il totale delle osservazioni.

In [11]:
# Probabilità marginale:
prob_gender = df['gender'].value_counts() / len(df)
prob_productid = df['productid'].value_counts() / len(df)
print("\nProbabilità marginale di genere:\n", prob_gender)
print("\nProbabilità marginale di prodotto scelto:\n", prob_productid)


Probabilità marginale di genere:
 gender
1    0.553878
2    0.446122
Name: count, dtype: float64

Probabilità marginale di prodotto scelto:
 productid
2    0.444082
1    0.336735
3    0.219184
Name: count, dtype: float64


# Test del chi-quadrato

In [12]:
# Ricalcolo la tabelle di contingenza SENZA MARGINI
contingency_table = pd.crosstab(df['gender'], df['productid'])

In [13]:
# Test chi square: utilizzo i -1 per non prendere l'ultima riga e l'ultima colonna
chi2, p, _, expected = chi2_contingency(contingency_table)

print("\nValore chi2:", chi2)
print("P-value:", p)
print("\nTabella delle frequenze attese:\n", pd.DataFrame(expected, columns=contingency_table.columns, index=contingency_table.index))



Valore chi2: 12.569256167968339
P-value: 0.0018647503713553623

Tabella delle frequenze attese:
 productid          1           2           3
gender                                      
1          456.94898  602.618776  297.432245
2          368.05102  485.381224  239.567755


## Lo Scostamento Percentuale

Calcoliamo quindi Lo Scostamento Percentuale tra Frequenza Osservata e Attesa

In [14]:
print(100.0 * (contingency_table.values - expected) / expected)

[[  8.32719234  -2.09398977  -8.55060113]
 [-10.33851784   2.59976589  10.61588814]]


## Resti Standardizzati

In [15]:
# Questa funzione calcola i resti standardizzati
def stdres(observed, expected):
    n = observed.sum()
    rsum, csum = margins(observed)
    # With integers, the calculation
    #     csum * rsum * (n - rsum) * (n - csum)
    # might overflow, so convert rsum and csum to floating point.
    rsum = rsum.astype(np.float64)
    csum = csum.astype(np.float64)
    v = csum * rsum * (n - rsum) * (n - csum) / n**3
    return (observed - expected) / np.sqrt(v)

residuals = stdres(contingency_table.values, expected)
print("\nStandardized Residuals:\n", residuals)


Standardized Residuals:
 [[ 3.27236525 -1.03219876 -2.49855659]
 [-3.27236525  1.03219876  2.49855659]]
